# * Sales Channel

In [2]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [9]:
''' Input parameter '''

op_dir = 'data'
op_sales_file = f'sales_channel_{str_curr_dt}'

# v_year = 0
# v_month_start = 0
# v_month_end = 0
# v_date = 20250101

print(f'\nParameter input...\n')
print(f'   -> op_dir: {op_dir}')
print(f'   -> op_sales_file: {op_sales_file}')
# print(f'\n   -> v_year: {v_year}')
# print(f'   -> v_month_start: {v_month_start}')
# print(f'   -> v_month_end: {v_month_end}')
# print(f'\n   -> v_date: {v_date}')


Parameter input...

   -> op_dir: data
   -> op_sales_file: sales_channel_20251124


## Import Transaction
-> DASH_PERFORMANCE

In [15]:
''' Execute transaction '''


# Input parameter
v_start_date = 20250101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()

query = (f"""
    SELECT /*+PARALLEL(8)*/ 
        TM_KEY_MTH, COMPANY, PRODUCT, SUB_PRODUCT, GP_SKU, SPECIAL_SKU, PROJECT_GROUP, CUST_TYPE, BU_CHANNEL, TDS_CHANNEL, TDS_SUB_CHANNEL, GROUP_TDS_SPECIAL, TDS_SPECIAL_CHANNEL, CHANNEL_GROUP, BUNDLE_TYPE
        , ORGID_H, HOP_HINT, D_CLUSTER, ORGID_G, TDS_GMD, TDS_SGMD, DISTRICT_TH, DISTRICT_EN
        , A.CCAA, O.PROVINCE_CD, O.PROVINCE_ENG, O.PROVINCE_TH
        , TARGET, ACTUAL, RUNRATE, TARGET_VALUE, ACTIVATION_VALUE, ACTIVATION_VALUE_RUNRATE
        
    FROM (
        SELECT SUBSTR(TM_KEY_DAY, 1, 6) TM_KEY_MTH
            , COMPANY, PRODUCT, SUB_PRODUCT, GP_SKU, SPECIAL_SKU, PROJECT_GROUP, CUST_TYPE, BU_CHANNEL, TDS_CHANNEL, TDS_SUB_CHANNEL, GROUP_TDS_SPECIAL, TDS_SPECIAL_CHANNEL, CHANNEL_GROUP, BUNDLE_TYPE 
            , ORGID_H, HOP_HINT
            , D_CLUSTER
            , ORGID_G, TDS_GMD, TDS_SGMD
            , DISTRICT_TH, DISTRICT_EN
            , SUBSTR(PARTNER_CCAATT, 1, 4) AS CCAA --, PARTNER_CCAATT, PARTNER_SCAB
            , SUM(TARGET) TARGET, SUM(ACTUAL) ACTUAL, SUM(RUNRATE) RUNRATE, SUM(TARGET_VALUE) TARGET_VALUE, SUM(ACTIVATION_VALUE) ACTIVATION_VALUE, SUM(ACTIVATION_VALUE_RUNRATE) ACTIVATION_VALUE_RUNRATE
            
        FROM CDSSHRE.DASH_PERFORMANCE
        WHERE TM_KEY_DAY >= {v_start_date}
        AND CUST_TYPE = 'INDY'

        GROUP BY SUBSTR(TM_KEY_DAY, 1, 6) 
            , COMPANY, PRODUCT, SUB_PRODUCT, GP_SKU, SPECIAL_SKU, PROJECT_GROUP, CUST_TYPE, BU_CHANNEL, TDS_CHANNEL, TDS_SUB_CHANNEL, GROUP_TDS_SPECIAL, TDS_SPECIAL_CHANNEL, CHANNEL_GROUP, BUNDLE_TYPE 
            , ORGID_H, HOP_HINT
            , D_CLUSTER
            , ORGID_G, TDS_GMD, TDS_SGMD
            , DISTRICT_TH, DISTRICT_EN
            , SUBSTR(PARTNER_CCAATT, 1, 4)
    ) A

    LEFT JOIN (
        SELECT DISTINCT SUBSTR(CCAATT,1,4) AS CCAA--, DISTRICT_EN
            , SUBSTR(CCAATT,1,2) AS PROVINCE_CD, PROVINCE_ENG, PROVINCE_TH
        FROM CDSAPPO.DIM_MOOC_AREA
        WHERE TEAM_CODE <> 'ไม่ระบุ' AND REMARK <> 'Dummy'
    ) O
        ON O.CCAA = A.CCAA
""")


try:
    # Create Dataframe
    src_cur.execute(query)
    rows = src_cur.fetchall()
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\nDataFrame: {src_df.shape[0]} rows, {src_df.shape[1]} columns')

    # Generate CSV file
    src_df.to_csv(f'{op_dir}/{op_sales_file}.csv', index=False, encoding='utf-8')
    print(f'\n   -> Generate "{op_sales_file}.csv" successfully')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20250101

Data as of 2025-11-24, 14:10:23

DataFrame: 2023445 rows, 33 columns

   -> Generate "sales_channel_20251124.csv" successfully


## Review

In [16]:
src_df

,TM_KEY_MTH,COMPANY,PRODUCT,SUB_PRODUCT,GP_SKU,SPECIAL_SKU,PROJECT_GROUP,CUST_TYPE,BU_CHANNEL,TDS_CHANNEL,...,CCAA,PROVINCE_CD,PROVINCE_ENG,PROVINCE_TH,TARGET,ACTUAL,RUNRATE,TARGET_VALUE,ACTIVATION_VALUE,ACTIVATION_VALUE_RUNRATE
0,202506,DTAC,PREPAY,PREPAY,MIGRANT,MIGRANT,None,INDY,DTAC Channel,Unidentified,...,1206,12,Nonthaburi,นนทบุรี,0.0,189,189.0,0.0,0.00,0.00
1,202506,TRUE,PREPAY,AP,MASS,MASS,MASS,INDY,Non-Shop,Wholesales,...,6701,67,Phetchabun,เพชรบูรณ์,0.0,410,410.0,0.0,0.00,0.00
2,202506,TRUE,PREPAY,AP,MASS,MASS,MASS,INDY,Non-Shop,Wholesales,...,4703,47,Sakon Nakhon,สกลนคร,0.0,86,86.0,0.0,0.00,0.00
3,202506,TRUE,PREPAY,INFLOW M1,MASS,MASS,MASS,INDY,7-11,7-11,...,5015,50,Chiang Mai,เชียงใหม่,0.0,0,0.0,0.0,18399.58,18399.58
4,202506,TRUE,PREPAY,PREPAY REV SUBS,MASS,MASS,MASS,INDY,Non-Shop,Wholesales,...,2406,24,Chachoengsao,ฉะเชิงเทรา,0.0,192,192.0,0.0,44622.56,44622.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023440,202504,TRUE,POSTPAY,POSTPAY,SIM Only,SIM Only,A0-MASS,INDY,Retail Shop,Retail Shop,...,3415,34,Ubon Ratchathani,อุบลราชธานี,0.0,17,17.0,0.0,10483.00,10483.00
2023441,202504,TRUE,PREPAY,APD1_200UP,MASS,MASS,MASS,INDY,Non-Shop,Lotus Non Shop,...,6509,65,Phitsanulok,พิษณุโลก,0.0,0,0.0,0.0,0.00,0.00
2023442,202504,DTAC,PREPAY,INFLOW M1,TOURIST,TOURIST,None,INDY,DTAC Channel,Unidentified,...,4104,41,Udon Thani,อุดรธานี,0.0,0,0.0,0.0,135.52,135.52
2023443,202504,TRUE,PREPAY,APD1,MASS,MASS,MASS,INDY,7-11,7-11,...,3010,30,Nakhon Ratchasima,นครราชสีมา,0.0,101,101.0,0.0,0.00,0.00
